# Import Libs

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.optim import Adam, lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import models, transforms
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
import pandas as pd
import numpy  as np
import time
import os

# Configuration

In [49]:
classes = [
    "angry", 
    "disgust", 
    "fear", 
    "happy", 
    "sad", 
    "surprise", 
    "neutral"
]

In [50]:
class Config:
    # dataset
    TRAIN_DS_PATH = './dataset/train.csv'
    VAL_DS_PATH = './dataset/val.csv'
    TEST_DS_PATH = './dataset/finaltest.csv'
    
    # images dir
    TRAIN_IMG_DIR = './dataset/train/'
    VAL_IMG_DIR  = './dataset/val/'
    TEST_IMG_DIR  = './dataset/finaltest/'
    
    # training hyperparams
    EPOCHS = 50
    LR = 1e-2
    BATCH_SIZE = 8
    NUM_WORKERS = 0
    SHUFFLE = True
    
    # saved model path
    MODEL_DIR = './model/'

In [51]:
cfg = Config()

In [52]:
# tensorboard writer
writer = SummaryWriter("runs")

# Dataset

In [53]:
# reference: https://github.com/omarsayed7/Deep-Emotion/blob/master/data_loaders.py
class EmotionDataset(Dataset):
    def __init__(self, csv_file, img_dir, datatype, transform):
        '''
        Pytorch Dataset class
        params:-
                 csv_file : the path of the csv file    (train, validation, test)
                 img_dir  : the directory of the images (train, validation, test)
                 datatype : data type for the dataset   (train, val, test)
                 transform: pytorch transformation over the data
        return :-
                 image, labels
        '''
        self.csv_file = pd.read_csv(csv_file)
        self.labels = self.csv_file['emotion']
        self.img_dir = img_dir
        self.datatype = datatype
        self.transform = transform

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img = Image.open(self.img_dir + self.datatype + str(idx) + '.jpg').convert('RGB')
        labels = np.array(self.labels[idx])
        labels = torch.from_numpy(labels).long()

        if self.transform :
            img = self.transform(img)
        return img, labels

In [54]:
# tranformations
transformation= transforms.Compose([
    transforms.Resize(256),
#     transforms.Grayscale(3), # no need this since you .convert("RGB") at __getitem__
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [55]:
# create datasets
train_dataset = EmotionDataset(
    csv_file=cfg.TRAIN_DS_PATH, 
    img_dir=cfg.TRAIN_IMG_DIR,
    datatype='train',
    transform=transformation
)

validation_dataset = EmotionDataset(
    csv_file=cfg.VAL_DS_PATH, 
    img_dir=cfg.VAL_IMG_DIR,
    datatype='val',
    transform = transformation
)


test_dataset = EmotionDataset(
    csv_file=cfg.TEST_DS_PATH, 
    img_dir=cfg.TEST_IMG_DIR,
    datatype='finaltest',
    transform = transformation
)

In [56]:
# create data loaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

val_loader = DataLoader(
    validation_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=cfg.BATCH_SIZE, 
    shuffle =cfg.SHUFFLE, 
    num_workers=cfg.NUM_WORKERS
)

# Helpers

In [57]:
def train_epoch(epoch, model, train_loader, criterion, optimizer, device, steps_print_loss=0, writer=None):
    """Train the model for 1 epoch
    Args:
        epoch: Current training epoch
        model: nn.Module
        train_loader: train DataLoader
        criterion: callable loss function
        optimizer: pytorch optimizer
        device: torch.device
        steps_print_loss: loss will print out in every specified steps
        writer: Tensorboard SummaryWriter
    Returns
    -------
    Tuple[Float, Float]
        average train loss and average train accuracy for current epoch
    """
    train_losses = []
    train_corrects = []
    model.train()

    # Iterate over data.
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # prediction
        outputs = model(inputs)

        # calculate loss
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # statistics
        train_losses.append(loss.item())
        train_corrects.append(torch.sum(preds == labels.data).item())
        
        if steps_print_loss and (batch_idx % steps_print_loss == 0):
            print(f"loss: {loss.item():>7f}  [{batch_idx:>5d}/{len(train_loader):>5d}]")
        
        if writer:
            writer.add_scalar(
                "train_loss", 
                sum(train_losses)/len(train_losses),
                global_steps=epoch*len(train_loader)*batch_idx
            )
            writer.add_scalar(
                "train_accuracy",
                sum(train_corrects)/len(train_loader.dataset),
                global_steps=epoch*len(train_loader)*batch_idx
            )
        
    ave_train_loss = sum(train_losses)/len(train_losses)
    ave_train_accuracy = sum(train_corrects)/len(train_loader.dataset)      

    return ave_train_loss, ave_train_accuracy


def val_epoch(epoch, model, val_loader, criterion, device, writer=None):
    """Validate the model for 1 epoch
    Args:
        epoch: Current validation epoch
        model: nn.Module
        val_loader: val DataLoader
        criterion: callable loss function
        device: torch.device
        writer: Tensorboard SummaryWriter
    Returns
    -------
    Tuple[Float, Float]
        average val loss and average val accuracy for current epoch
    """

    val_losses = []
    val_corrects = []
    model.eval()

    # Iterate over data
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # prediction
            outputs = model(inputs)

            # calculate loss
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # statistics
            val_losses.append(loss.item())
            val_corrects.append(torch.sum(preds == labels.data).item())
            
            if writer:
                writer.add_scalar(
                    "val_loss", 
                    sum(val_losses)/len(val_losses),
                    global_steps=epoch*len(val_loader)*batch_idx
                )
                writer.add_scalar(
                    "val_accuracy",
                    sum(val_corrects)/len(val_loader.dataset),
                    global_steps=epoch*len(val_loader)*batch_idx
                )
            
        ave_val_loss = sum(val_losses)/len(val_losses)
        ave_val_accuracy = sum(val_corrects)/len(val_loader.dataset)

    return ave_val_loss, ave_val_accuracy

# Training and Evaluating Model

In [58]:
# load pretrained model
model = models.mobilenet.mobilenet_v2(pretrained=True)

In [59]:
# freeze all layers
for param in model.parameters():
    param.requires_grad = False

num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, len(classes))

In [60]:
# transfer to cuda device if any
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using {device}")

Using cuda


In [61]:
# loss, optimizer and scheduler 
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=cfg.LR)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

In [62]:
# Start training and val loops

since = time.time()
for epoch in range(cfg.EPOCHS):
    print(f"Epoch {epoch+1}\n-------------------------------")
    
    # Train
    ave_train_loss, ave_train_acc = train_epoch(epoch, model, train_loader, criterion, optimizer, device, 500, writer)
    print(f'Average TrainLoss: {ave_train_loss:.4f} \tAverage TrainAcc: {ave_train_acc:.4f}')
    
    # Evaluation
    print("Evaluation:")
    ave_val_loss, ave_val_acc = val_epoch(epoch, model, val_loader, criterion, device, writer)
    print(f'Average ValLoss: {ave_val_loss:.4f} \tAverage ValAcc: {ave_val_acc:.4f}')
    
    # log metrics at every epoch
#     if writer:
#         writer.add_scalar("train_loss", ave_train_loss, epoch)
#         writer.add_scalar("train_accuracy", ave_train_acc, epoch)
#         writer.add_scalar("val_loss", ave_val_loss, epoch)
#         writer.add_scalar("val_accuracy", ave_val_acc, epoch)
    
    # schedule lr
    scheduler.step(val_loss)
    print("\n")
        
time_elapsed = time.time() - since
print(f"Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s")

Epoch 1
-------------------------------
loss: 2.140212  [    0/ 3589]
loss: 2.059390  [  500/ 3589]
loss: 2.356801  [ 1000/ 3589]
loss: 9.660825  [ 1500/ 3589]
loss: 2.275562  [ 2000/ 3589]
loss: 3.315902  [ 2500/ 3589]
loss: 3.490290  [ 3000/ 3589]
loss: 2.930248  [ 3500/ 3589]
Average TrainLoss: 3.6870 	Average TrainAcc: 0.2584
Evaluation:
Average ValLoss: 3.3211 	Average ValAcc: 0.2856


Epoch 2
-------------------------------
loss: 4.368715  [    0/ 3589]
loss: 5.648328  [  500/ 3589]
loss: 3.797549  [ 1000/ 3589]
loss: 4.148750  [ 1500/ 3589]
loss: 1.285423  [ 2000/ 3589]
loss: 3.271886  [ 2500/ 3589]
loss: 3.513383  [ 3000/ 3589]
loss: 3.547631  [ 3500/ 3589]
Average TrainLoss: 3.9700 	Average TrainAcc: 0.2653
Evaluation:
Average ValLoss: 2.5226 	Average ValAcc: 0.3346


Epoch 3
-------------------------------
loss: 1.579693  [    0/ 3589]
loss: 6.319479  [  500/ 3589]
loss: 3.984198  [ 1000/ 3589]
loss: 1.539312  [ 1500/ 3589]
loss: 3.414398  [ 2000/ 3589]
loss: 6.989782  [ 2500

Average ValLoss: 1.5408 	Average ValAcc: 0.4062


Epoch 22
-------------------------------
loss: 1.395439  [    0/ 3589]
loss: 2.170592  [  500/ 3589]
loss: 1.850816  [ 1000/ 3589]
loss: 1.356704  [ 1500/ 3589]
loss: 2.013103  [ 2000/ 3589]
loss: 1.604768  [ 2500/ 3589]
loss: 1.258372  [ 3000/ 3589]
loss: 2.230244  [ 3500/ 3589]
Average TrainLoss: 1.7378 	Average TrainAcc: 0.3402
Evaluation:
Average ValLoss: 1.5347 	Average ValAcc: 0.4046


Epoch 23
-------------------------------
loss: 2.086190  [    0/ 3589]
loss: 1.687603  [  500/ 3589]
loss: 1.498874  [ 1000/ 3589]
loss: 1.533374  [ 1500/ 3589]
loss: 2.006775  [ 2000/ 3589]
loss: 1.365455  [ 2500/ 3589]
loss: 1.793460  [ 3000/ 3589]
loss: 1.996094  [ 3500/ 3589]
Average TrainLoss: 1.7446 	Average TrainAcc: 0.3332
Evaluation:
Average ValLoss: 1.6488 	Average ValAcc: 0.3564


Epoch 24
-------------------------------
loss: 2.042656  [    0/ 3589]
loss: 1.326475  [  500/ 3589]
loss: 2.111315  [ 1000/ 3589]
loss: 2.155059  [ 1500/ 3589]

loss: 1.505315  [ 3500/ 3589]
Average TrainLoss: 1.7315 	Average TrainAcc: 0.3396
Evaluation:
Average ValLoss: 1.6559 	Average ValAcc: 0.3943


Epoch 43
-------------------------------
loss: 2.218401  [    0/ 3589]
loss: 1.213001  [  500/ 3589]
loss: 1.481663  [ 1000/ 3589]
loss: 1.845937  [ 1500/ 3589]
loss: 1.708820  [ 2000/ 3589]
loss: 1.427733  [ 2500/ 3589]
loss: 1.373033  [ 3000/ 3589]
loss: 1.969328  [ 3500/ 3589]
Average TrainLoss: 1.7205 	Average TrainAcc: 0.3417
Evaluation:
Average ValLoss: 1.5795 	Average ValAcc: 0.3912


Epoch 44
-------------------------------
loss: 1.677276  [    0/ 3589]
loss: 1.191614  [  500/ 3589]
loss: 1.620971  [ 1000/ 3589]
loss: 2.060056  [ 1500/ 3589]
loss: 1.371291  [ 2000/ 3589]
loss: 1.302285  [ 2500/ 3589]
loss: 1.915738  [ 3000/ 3589]
loss: 2.127298  [ 3500/ 3589]
Average TrainLoss: 1.7321 	Average TrainAcc: 0.3378
Evaluation:
Average ValLoss: 1.5566 	Average ValAcc: 0.4007


Epoch 45
-------------------------------
loss: 1.339363  [    0/ 3

# Evaluating on Unseen Test Data

In [63]:
test_losses = []
test_corrects = []

model.eval()
# Iterate over data
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # prediction
        outputs = model(inputs)

        # calculate loss
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # statistics
        test_losses.append(loss.item())
        test_corrects.append(torch.sum(preds == labels.data).item())

    ave_test_loss = sum(test_losses)/len(test_losses)
    ave_test_acc = sum(test_corrects)/len(test_loader.dataset)
    
print(f'Average TestLoss: {ave_test_loss:.4f} \tAverage TestAcc: {ave_test_acc:.4f}')

Average TestLoss: 1.5070 	Average TestAcc: 0.4138


# Saving model

In [64]:
# Save the model
model_name = f"model_epoch{cfg.EPOCHS}_lr{cfg.LR}_batch{cfg.BATCH_SIZE}.pt"
torch.save(model.state_dict(), cfg.MODEL_DIR + model_name)